## Instacart Analysis – Product Review
1.	Import libraries, set directory paths & import data
2.	Check data frame dimensions, columns and datatypes
3.	Categorise product pricing
    -	Review descriptive statistic
    -	Assign pricing category (low range, medium range, high range)
    -	Check value counts
4.	Address null values for price and price category
    -	Save null group to subset to assess which products are affected
    -	Assign to low range pricing group
    -	Calculate average price for low range products
    -	Impute mean values in place of nulls
5.	Review of popular products
    -	Count total number of unique products
    -	Calculate total number of orders per product
    -	Review descriptive statistics for assigning popular product value
    -	Assign top_order flag for products ordered > 200,000 times
    -	Create list of department_id associated with top_order products
6.	Review of product generation
    -	Calculate total generated revenue
    -	Calculate total revenue per product
    -	Review product_revenue descriptive statistics
    -	Assign flag to big_revenue products
    -	Create list of department_id associated with big_revenue products
7.	Review key departments based on product orders and revenue
    -	Compare dept ID from orders and revenue lists
    -	Combine the list (unique values only)
    -	Review top 7 departments (out of 21 departments)
8.	Assess Key Departments & mark key_dept
9.	Export new data to pickle file


### import libraries

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

### set data set directory path

In [2]:
datasetpath = r'D:\My Documents\! Omnicompetent Ltd\Courses\Career Foundry - Data Analytics\Data Analytics Course\Instacart Basket Analysis\02 Data Sets'
datasetpath

'D:\\My Documents\\! Omnicompetent Ltd\\Courses\\Career Foundry - Data Analytics\\Data Analytics Course\\Instacart Basket Analysis\\02 Data Sets'

### set visualisation directory path

In [3]:
vizpath = r'D:\My Documents\! Omnicompetent Ltd\Courses\Career Foundry - Data Analytics\Data Analytics Course\Instacart Basket Analysis\04 Analysis'
vizpath

'D:\\My Documents\\! Omnicompetent Ltd\\Courses\\Career Foundry - Data Analytics\\Data Analytics Course\\Instacart Basket Analysis\\04 Analysis'

### import flagged data set

In [4]:
df_testing = pd.read_pickle(os.path.join(datasetpath,'testing_sample_keep.pkl'))
df_testing.head()

,order_id,user_id,number_of_orders,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,reordered,product_name,department_id,price,gender,state,age,n_dependants,marital_status,income,region,max_order
0,2539329,1,1,2,8,0.0,196,0,Soda,7,9.0,Female,Alabama,31,3,married,40423,South,10
1,473747,1,3,3,12,21.0,196,1,Soda,7,9.0,Female,Alabama,31,3,married,40423,South,10
2,2254736,1,4,4,7,29.0,196,1,Soda,7,9.0,Female,Alabama,31,3,married,40423,South,10
3,550135,1,7,1,9,20.0,196,1,Soda,7,9.0,Female,Alabama,31,3,married,40423,South,10
4,2539329,1,1,2,8,0.0,14084,0,Organic Unsweetened Vanilla Almond Milk,16,12.5,Female,Alabama,31,3,married,40423,South,10


### review dimensions, columns & datatypes

In [5]:
df_testing.shape

(9268148, 19)

In [6]:
df_testing.dtypes

order_id                    int64
user_id                     int64
number_of_orders            int64
order_day_of_week           int64
order_hour_of_day           int64
days_since_prior_order    float64
product_id                  int64
reordered                   int64
product_name               object
department_id               int64
price                     float64
gender                     object
state                      object
age                         int64
n_dependants                int64
marital_status             object
income                      int64
region                     object
max_order                   int64
dtype: object

## Categorize product prices

### product price descriptive statistics

In [7]:
df_testing['price'].describe()

count    9.266677e+06
mean     7.789125e+00
std      4.239684e+00
min      1.000000e+00
25%      4.200000e+00
50%      7.400000e+00
75%      1.130000e+01
max      2.500000e+01
Name: price, dtype: float64

### assign product pricing flag
    Low range product:     <5USD
    Medium range product:  >=5USD <12USD
    High range product:    >=12USD

In [8]:
df_testing.loc[(df_testing['price'] <5), 'prod_price_range'] = 'Low Range Product'

In [9]:
df_testing.loc[(df_testing['price'] >=5) & (df_testing['price'] <12), 'prod_price_range'] = 'Medium Range Product'

In [10]:
df_testing.loc[(df_testing['price'] >=12), 'prod_price_range'] = 'High Range Product'

### review prod_price_range value counts

In [11]:
df_testing['prod_price_range'].value_counts(dropna=False)

Medium Range Product    4549280
Low Range Product       2831726
High Range Product      1885671
NaN                        1471
Name: prod_price_range, dtype: int64

### save null values product to subset for review

In [12]:
df_null = df_testing[df_testing['price'].isnull()]
df_null.head()

,order_id,user_id,number_of_orders,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,reordered,product_name,department_id,price,gender,state,age,n_dependants,marital_status,income,region,max_order,prod_price_range
6763,1578338,3819,6,2,13,30.0,21553,1,Lowfat 2% Milkfat Cottage Cheese,16,NaN,Male,Utah,69,2,married,129749,West,10,NaN
9143,2977209,5218,8,4,15,12.0,21553,1,Lowfat 2% Milkfat Cottage Cheese,16,NaN,Female,Iowa,29,2,married,120566,Midwest,19,NaN
9144,2203682,5218,10,5,11,2.0,21553,1,Lowfat 2% Milkfat Cottage Cheese,16,NaN,Female,Iowa,29,2,married,120566,Midwest,19,NaN
19214,580648,10662,13,3,11,9.0,21553,0,Lowfat 2% Milkfat Cottage Cheese,16,NaN,Male,Arizona,45,3,married,147917,West,19,NaN
24616,3175959,13460,5,3,14,0.0,21553,1,Lowfat 2% Milkfat Cottage Cheese,16,NaN,Male,Virginia,73,1,married,215922,South,5,NaN


### count & list products affected by null pricing value

In [13]:
len(df_null['product_name'].unique())

2

In [14]:
df_null['product_name'].value_counts()

Lowfat 2% Milkfat Cottage Cheese    1274
2 % Reduced Fat  Milk                197
Name: product_name, dtype: int64

### place null value products into 'Low Price Range' product

In [15]:
df_testing.loc[(df_testing['price'].isnull()), 'prod_price_range'] = 'Low Range Product'

### recheck value counts

In [16]:
df_testing['prod_price_range'].value_counts(dropna=False)

Medium Range Product    4549280
Low Range Product       2833197
High Range Product      1885671
Name: prod_price_range, dtype: int64

### copy value counts to clipboard

In [17]:
prodpricerange = df_testing['prod_price_range'].value_counts()
prodpricerange.to_clipboard()

### create subset for low range products

In [18]:
df_low = df_testing[df_testing['prod_price_range']=='Low Range Product']

In [19]:
df_low['prod_price_range'].value_counts()

Low Range Product    2833197
Name: prod_price_range, dtype: int64

### calculate average price for low range product

In [20]:
df_low['price'].mean()

2.978849754530339

### populate null value prices with 2.98USD

In [21]:
df_testing.loc[(df_testing['price'].isnull()), 'price'] = 2.98

### check for null again

In [22]:
df_testing[df_testing['price'].isnull()]

,order_id,user_id,number_of_orders,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,reordered,product_name,department_id,price,gender,state,age,n_dependants,marital_status,income,region,max_order,prod_price_range


## Review of popular products

### total number of different products

In [23]:
len(df_testing['product_name'].unique())

48163

### calculate total number of orders per product

In [24]:
df_testing['sum_product_order'] = df_testing.groupby(['product_id']) ['number_of_orders'].transform(np.sum)

### review sum_product_order descriptive statistics

In [25]:
df_testing['sum_product_order'].describe()

count    9.268148e+06
mean     1.957983e+05
std      4.375462e+05
min      1.000000e+00
25%      8.140000e+03
50%      3.443500e+04
75%      1.485820e+05
max      2.432147e+06
Name: sum_product_order, dtype: float64

### create subset for items ordered more than 200,000 times

In [26]:
df_prodpop = df_testing[df_testing['sum_product_order']>200000]

### total number of unique products for popular ordering

In [27]:
len(df_prodpop['product_name'].unique())

80

In [28]:
df_prodpop['product_name'].value_counts()

Banana                      135820
Bag of Organic Bananas      109297
Organic Strawberries         77137
Organic Baby Spinach         69552
Organic Hass Avocado         62180
                             ...  
Organic Broccoli             11027
100% Raw Coconut Water       10973
Organic Black Beans          10942
Organic Reduced Fat Milk     10362
Organic Carrot Bunch         10126
Name: product_name, Length: 80, dtype: int64

#### 80 products have been ordered 200,000 times

### assign popular product flag: 'top_order'

In [33]:
df_testing.loc[(df_testing['sum_product_order'] >200000), 'top_order'] = 1

In [34]:
df_testing.loc[(df_testing['sum_product_order'] <=200000), 'top_order'] = 0

In [35]:
df_testing[['sum_product_order','top_order']].head()

,sum_product_order,top_order
0,181305,0.0
1,181305,0.0
2,181305,0.0
3,181305,0.0
4,96579,0.0


### check top_order value counts

In [37]:
df_testing['top_order'].value_counts(dropna=False)

0.0    7327760
1.0    1940388
Name: top_order, dtype: int64

### list department_id from df_prodpop, top order

In [38]:
df_prodpop.groupby(['department_id']).count()

,order_id,user_id,number_of_orders,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,reordered,product_name,price,gender,state,age,n_dependants,marital_status,income,region,max_order,prod_price_range,sum_product_order
department_id,,,,,,,,,,,,,,,,,,,,
1,15836,15836,15836,15836,15836,15836,15836,15836,15836,15836,15836,15836,15836,15836,15836,15836,15836,15836,15836,15836
3,17468,17468,17468,17468,17468,17468,17468,17468,17468,17468,17468,17468,17468,17468,17468,17468,17468,17468,17468,17468
4,1569732,1569732,1569732,1569732,1569732,1569732,1569732,1569732,1569732,1569732,1569732,1569732,1569732,1569732,1569732,1569732,1569732,1569732,1569732,1569732
7,74781,74781,74781,74781,74781,74781,74781,74781,74781,74781,74781,74781,74781,74781,74781,74781,74781,74781,74781,74781
12,14221,14221,14221,14221,14221,14221,14221,14221,14221,14221,14221,14221,14221,14221,14221,14221,14221,14221,14221,14221
13,14269,14269,14269,14269,14269,14269,14269,14269,14269,14269,14269,14269,14269,14269,14269,14269,14269,14269,14269,14269
15,10942,10942,10942,10942,10942,10942,10942,10942,10942,10942,10942,10942,10942,10942,10942,10942,10942,10942,10942,10942
16,190697,190697,190697,190697,190697,190697,190697,190697,190697,190697,190697,190697,190697,190697,190697,190697,190697,190697,190697,190697
20,32442,32442,32442,32442,32442,32442,32442,32442,32442,32442,32442,32442,32442,32442,32442,32442,32442,32442,32442,32442


### create list of dept ids for later use

In [39]:
deptpop = [1,3,4,7,12,13,15,16,20]
deptpop

[1, 3, 4, 7, 12, 13, 15, 16, 20]

## Review of product revenue generation

### calculate total revenue for the full data set (to help get perspective)

In [40]:
total_rev = df_testing['price'].sum()
total_rev

72183689.77999997

### groupby 'product_id' and calculate total revenue per product

In [41]:
df_testing['product_revenue'] = df_testing.groupby(['product_id']) ['price'].transform(np.sum)

In [42]:
df_testing[['product_name','price','product_revenue']].head()

,product_name,price,product_revenue
0,Soda,9.0,91152.0
1,Soda,9.0,91152.0
2,Soda,9.0,91152.0
3,Soda,9.0,91152.0
4,Organic Unsweetened Vanilla Almond Milk,12.5,58687.5


### review product_revenue descriptive statistics

In [43]:
df_testing['product_revenue'].describe()

count    9.268148e+06
mean     9.135354e+04
std      2.501206e+05
min      1.000000e+00
25%      2.724400e+03
50%      1.264860e+04
75%      5.361830e+04
max      1.670586e+06
Name: product_revenue, dtype: float64

### create subset for product generating over $100,000

In [44]:
df_prodrev = df_testing[df_testing['product_revenue'] > 100000]

### count number of products that contribute > $100,000

In [45]:
len(df_prodrev['product_name'].unique())

67

#### 67 products that contribute > $100,000 revenue

### assign flag to big_revenue products

In [46]:
df_testing.loc[(df_testing['product_revenue'] >100000), 'big_revenue'] = 1

In [47]:
df_testing.loc[(df_testing['product_revenue'] <=100000), 'big_revenue'] = 0

### check new columns

In [48]:
df_testing[['product_name','product_revenue','big_revenue']].head(20)

,product_name,product_revenue,big_revenue
0,Soda,91152.0,0.0
1,Soda,91152.0,0.0
2,Soda,91152.0,0.0
3,Soda,91152.0,0.0
4,Organic Unsweetened Vanilla Almond Milk,58687.5,0.0
5,Original Beef Jerky,8166.4,0.0
6,Original Beef Jerky,8166.4,0.0
7,Original Beef Jerky,8166.4,0.0
8,Aged White Cheddar Popcorn,3351.1,0.0
9,Aged White Cheddar Popcorn,3351.1,0.0


### check big_revenue value counts

In [49]:
df_testing['big_revenue'].value_counts(dropna=False)

0.0    7714273
1.0    1553875
Name: big_revenue, dtype: int64

### count number of departments associated with big_revenue

In [50]:
len(df_prodrev['department_id'].unique())

6

### list depertment_id from top products for revenue in df_prodrev

In [51]:
df_prodrev.groupby(['department_id']).count()

,order_id,user_id,number_of_orders,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,reordered,product_name,price,...,age,n_dependants,marital_status,income,region,max_order,prod_price_range,sum_product_order,top_order,product_revenue
department_id,,,,,,,,,,,,,,,,,,,,,
1,15836,15836,15836,15836,15836,15836,15836,15836,15836,15836,...,15836,15836,15836,15836,15836,15836,15836,15836,15836,15836
4,1260899,1260899,1260899,1260899,1260899,1260899,1260899,1260899,1260899,1260899,...,1260899,1260899,1260899,1260899,1260899,1260899,1260899,1260899,1260899,1260899
7,22773,22773,22773,22773,22773,22773,22773,22773,22773,22773,...,22773,22773,22773,22773,22773,22773,22773,22773,22773,22773
12,31294,31294,31294,31294,31294,31294,31294,31294,31294,31294,...,31294,31294,31294,31294,31294,31294,31294,31294,31294,31294
16,182390,182390,182390,182390,182390,182390,182390,182390,182390,182390,...,182390,182390,182390,182390,182390,182390,182390,182390,182390,182390
20,40683,40683,40683,40683,40683,40683,40683,40683,40683,40683,...,40683,40683,40683,40683,40683,40683,40683,40683,40683,40683


##### where:
    1 = frozen
    4 = produce
    7 = beverages
    12 = meat seafood
    16 = dairy eggs
    20 = deli

### check value counts for revenue depts & copy to clipboard

In [55]:
deptrev = df_prodrev['department_id'].value_counts()

In [56]:
deptrev.to_clipboard()

### create list containing dept IDs

In [57]:
deptrev = [1,4,7,12,16,20]

## Review popular departments

### compare dept IDs lists

In [58]:
deptpop

[1, 3, 4, 7, 12, 13, 15, 16, 20]

In [59]:
deptrev

[1, 4, 7, 12, 16, 20]

### combine department lists

In [60]:
dept_id = deptpop + list(set(deptrev) - set(deptpop))
dept_id

[1, 3, 4, 7, 12, 13, 15, 16, 20]

### revised list based on both groups
    workings shown in Client Profiling spreadsheet

In [61]:
keydept = [1,3,4,7,12,16,20]
keydept

[1, 3, 4, 7, 12, 16, 20]

## Assess Key departments

### set key_dept flag

In [62]:
df_testing.loc[df_testing['department_id'].isin(keydept), 'key_dept'] = 1

In [63]:
df_testing.loc[~df_testing['department_id'].isin(keydept), 'key_dept'] = 0

### check key_dept value_counts

In [65]:
df_testing['key_dept'].value_counts(dropna=False)

1.0    6508106
0.0    2760042
Name: key_dept, dtype: int64

### review columns

In [66]:
df_testing.dtypes

order_id                    int64
user_id                     int64
number_of_orders            int64
order_day_of_week           int64
order_hour_of_day           int64
days_since_prior_order    float64
product_id                  int64
reordered                   int64
product_name               object
department_id               int64
price                     float64
gender                     object
state                      object
age                         int64
n_dependants                int64
marital_status             object
income                      int64
region                     object
max_order                   int64
prod_price_range           object
sum_product_order           int64
top_order                 float64
product_revenue           float64
big_revenue               float64
key_dept                  float64
dtype: object

### export to new pickle file

In [67]:
df_testing.to_pickle(os.path.join(datasetpath,'testing_sample_prod.pkl'))

In [68]:
df_testing.shape

(9268148, 25)